In [1]:
import json
import os
from tqdm import tqdm

In [8]:
def dialogue_data_process(example, task_name, idx):
    """
    {
        "knowledge": "Fight Club is starring Jared Leto. Jared Leto starred in Panic Room", 
        "dialogue_history": "[Human]: Could you recommend some movies similar to Fight Club? ", 
        "right_response": "Sure, Panic Room is a similar movie", 
        "hallucinated_response": "Sure, Mr. Nobody is a similar movie."
    }
    """
    return {
        "task_name": task_name, 
        "idx": idx,
        "instruction": "{}\n{}".format(example["knowledge"], example["dialogue_history"]),
        "answer_positive": [example["right_response"]],
        "answer_negative": [example["hallucinated_response"]],
    }

In [9]:
def general_data_process(example, task_name, idx):
    """
    {
        "ID": "10", 
        "user_query": "Create a design for the given text snippet.\nUnited We Stand", 
        "chatgpt_response": "| U | N | I | T | E | D |\n| W | E |\n| S | T | A | N | D |", 
        "hallucination": "no", 
        "hallucination_spans": []
    }
    """
    return {
        "task_name": task_name, 
        "idx": idx,
        "instruction": "{}".format(example["user_query"]),
        "answer_positive": [example["chatgpt_response"]],
        "answer_negative": [example["hallucination"]],
    }

In [10]:
def qa_data_process(example, task_name, idx):
    """
    {
        "knowledge": " Donahue replaced Kelli Ward who resigned to run for the United States Senate.Kelli Ward (\"née\" Kaznoski; born January 25, 1969) is an American politician and osteopathic physician.", 
        "question": "Which  American politician did Donahue replaced ", 
        "right_answer": "Kelli Ward", 
        "hallucinated_answer": "Donahue replaced Kelli Ward's position."
    }
    """
    return {
        "task_name": task_name, 
        "idx": idx,
        "instruction": "{}\n{}".format(example["knowledge"], example["question"]),
        "answer_positive": [example["right_answer"]],
        "answer_negative": [example["hallucinated_answer"]],
    }

In [11]:
def summarization_data_process(example, task_name, idx):
    """
    {
        "document": " Donahue replaced Kelli Ward who resigned to run for the United States Senate.Kelli Ward (\"née\" Kaznoski; born January 25, 1969) is an American politician and osteopathic physician.", 
        "right_summary": "zxxxx", 
        "hallucinated_summary": "Kelli Ward", 
    }
    """
    return {
        "task_name": task_name, 
        "idx": idx,
        "instruction": "Document: {}\nQ: {}".format(example["document"], "Please summarize the document."),
        "answer_positive": [example["right_summary"]],
        "answer_negative": [example["hallucinated_summary"]],
    }

In [12]:
task_name_list = {
    "dialogue_data": dialogue_data_process,
    "general_data": general_data_process,
    "qa_data": qa_data_process,
    "summarization_data": summarization_data_process
}

In [16]:
all_data = list()

for task_name, processor in task_name_list.items():
    with open("{}.json".format(task_name), "r", encoding="utf-8") as fr:
        lines = fr.readlines()
        examples = [json.loads(i) for i in lines]
        for ei, example in enumerate(tqdm(examples)):
            example = processor(example, "{}-{}".format("HaluEval", task_name), ei + 1)
            all_data.append(example)
            

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 278963.77it/s]


In [17]:
all_data[12]

{'task_name': 'HaluEval-dialogue_data',
 'idx': 13,
 'instruction': "Wreck-It Ralph is starring Rich Moore\n[Human]: Could you recommend something like Wreck-It Ralph for me to watch with my kids? [Assistant]: Sure! Wreck-It Ralph was written by Rich Moore who also stars in it along side Sarah Silverman,its an animated film. Do you like Animated films? [Human]: Yes, animated films are what I'm looking for. [Assistant]: Another good Animated Film he directed is James and the Giant Peach. Have you seen this family movie yet? [Human]: I saw it many years ago, but I'd like to watch it again! ",
 'answer_positive': ['Great! You may also like the family movie Cinderella, have you seen this one yet?'],
 'answer_negative': ['Great! You may also like the family movie The Lion King, have you seen this one yet?']}

In [18]:
with open("halueval_dataset.json", "w", encoding="utf-8") as fw:
    for example in all_data:
        fw.write(json.dumps(example) + "\n")